In [37]:
import requests
import time
import json
user_key = '1a49911e9216b8037dc430f811fd5b6b'

# Organizations


In [3]:
r = requests.get('https://api.crunchbase.com/v/3/organizations?user_key={}'.format(user_key))

In [10]:
r.json()['data']['paging']

{u'current_page': 1,
 u'items_per_page': 100,
 u'next_page_url': u'https://api.crunchbase.com/v/3/organizations?page=2',
 u'number_of_pages': 4086,
 u'prev_page_url': None,
 u'sort_order': u'created_at DESC',
 u'total_items': 408546}

In [12]:
r.json()['data']['items'][0]

{u'properties': {u'api_path': u'organizations/fanarchy',
  u'city_name': None,
  u'country_code': None,
  u'created_at': 1448318261,
  u'domain': None,
  u'facebook_url': None,
  u'homepage_url': None,
  u'linkedin_url': None,
  u'name': u'Fanarchy',
  u'permalink': u'fanarchy',
  u'primary_role': u'company',
  u'profile_image_url': None,
  u'region_name': None,
  u'short_description': u'Win the game on the field. Change the game off it.',
  u'twitter_url': None,
  u'updated_at': 1448318261,
  u'web_path': u'organization/fanarchy'},
 u'type': u'OrganizationSummary',
 u'uuid': u'957ea17500ba9dea44b151f236e7920a'}

In [19]:
organizations = r.json()['data']['items']

In [40]:
len(organizations)

100

In [38]:
def get_json(url):
    r_ = requests.get(url)
    try:
        return r_.json()
    except ValueError:
        time.sleep(2)
        return get_json(url)

In [39]:
organizations = []
for i in xrange(1, 2):
    link = 'https://api.crunchbase.com/v/3/organizations?page={}&user_key={}'.format(i, user_key)
    if i%100 == 0:
        print 'page ', i, ' length is ',len(organizations)
    r_json = get_json(link)
    organizations += r_json['data']['items']
    time.sleep(0.1)

with open('tmp/organizations_json.json', 'w') as f:
    json.dump(organizations, f)

In [41]:
organizations[-1]

{u'properties': {u'api_path': u'organizations/livemail-customer-service-1-844-334-9858-phone-support-number-livemail-tech-support',
  u'city_name': None,
  u'country_code': None,
  u'created_at': 1448295806,
  u'domain': u'livemail.com',
  u'facebook_url': None,
  u'homepage_url': u'http://livemail.com',
  u'linkedin_url': None,
  u'name': u'Livemail Customer Service 1(844)-334-9858 Phone Support Number Livemail Tech Support',
  u'permalink': u'livemail-customer-service-1-844-334-9858-phone-support-number-livemail-tech-support',
  u'primary_role': u'company',
  u'profile_image_url': None,
  u'region_name': None,
  u'short_description': u'Livemail Customer Service 1(844)-334-9858 Phone Support Number Livemail Tech Support Livemail Customer Service 1(844)-334-9858 Phone Supprt',
  u'twitter_url': None,
  u'updated_at': 1448295881,
  u'web_path': u'organization/livemail-customer-service-1-844-334-9858-phone-support-number-livemail-tech-support'},
 u'type': u'OrganizationSummary',
 u'uuid'

# Relationships


In [43]:
relationships = ['founders', 'board_members_and_advisors', 'investors', 'owned_by', 'headquarters', 'offices', 'categories', 'competitors', 'funding_rounds', 'investments', 'acquisitions', 'acquired_by', 'ipo']

relationships_infos = {}
relationships_infos['person'] = ['bio', 'born_on', 'permalink']
relationships_infos['address'] = ['city', 'country']
relationships_infos['market'] = ['name']
relationships_infos['product'] = ['name']
relationships_infos['organization'] = ['permalink']
relationships_infos['fundinground'] = ['announced_on', 'funding_type', 'money_raised_usd']
relationships_infos['acquisition'] = ['announced_on']
relationships_infos['ipo'] = ['went_public_on']

In [44]:
organizations_dict = {}
for organization in [0]:
    infos = {}
    #permalink = organization['properties']['permalink']
    permalink = 'dropbox'
    for relation in relationships:
        time.sleep(0.5)
        url = 'https://api.crunchbase.com/v/3/organizations/{}/{}?user_key={}'.format(permalink, relation, user_key)
        r_json = get_json(url)
        items = r_json['data']['items' if 'items' in r_json['data'] else 'item']
        number_pages = r_json['data']['paging']['number_of_pages']
        next_page_url = r_json['data']['paging']['next_page_url']
        for j in xrange(2, number_pages + 1):
            url = next_page_url[:-1]+str(j)+'&user_key='+user_key
            r_json = get_json(url)
            items += r_json['data']['items']
        
        relation_items = []
        if type(items) is dict:
            items = [items]
        for item in items:
            if 'relationships' in item:
                for relationships_key in item['relationships']:
                    relationships_key = relationships_key.lower()
                    if relationships_key in relationships_infos:
                        item_temp = {}
                        for infos_key in relationships_infos[relationships_key]:
                            item_temp[infos_key] = item['relationships'][relationships_key]['properties'][infos_key]
                        relation_items.append(item_temp)
            relationships_key = item['type'].lower()
            if relationships_key in relationships_infos:
                item_temp = {}
                for infos_key in relationships_infos[relationships_key]:
                    item_temp[infos_key] = item['properties'][infos_key]
                relation_items.append(item_temp)
        infos[relation] = relation_items
    organizations_dict[permalink] = infos

with open('tmp/organizations_dict.json', 'w') as f:
    json.dump(organizations, f)

In [45]:
organizations_dict

{'dropbox': {'acquired_by': [],
  'acquisitions': [{'announced_on': u'2013-07-20'},
   {'announced_on': u'2014-03-17'},
   {'announced_on': u'2013-07-17'},
   {'announced_on': u'2013-09-01'},
   {'announced_on': u'2013-11-05'},
   {'announced_on': u'2014-03-28'},
   {'announced_on': u'2012-12-19'},
   {'announced_on': u'2012-02-27'},
   {'announced_on': u'2012-12-12'},
   {'announced_on': u'2013-03-15'},
   {'announced_on': u'2014-04-17'},
   {'announced_on': u'2014-04-17'},
   {'announced_on': u'2014-05-21'},
   {'announced_on': u'2014-06-05'},
   {'announced_on': u'2014-06-10'},
   {'announced_on': u'2014-06-16'},
   {'announced_on': u'2014-07-01'},
   {'announced_on': u'2013-11-17'},
   {'announced_on': u'2015-01-21'},
   {'announced_on': u'2015-01-26'},
   {'announced_on': u'2015-05-13'},
   {'announced_on': u'2015-07-22'}],
  'board_members_and_advisors': [{'bio': u'Based in San Francisco, Ali is an angel investor, startup advisor, and serial entrepreneur. Ali is also a co-founder